In [1]:
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

# Verileri yükleyin
train_features = np.load('train_features_stackedautoencoder.npy')
train_labels = np.load('train_labels_stackedautoencoder.npy')

# Pipeline oluşturun
pipe = Pipeline([
    ('scaler', StandardScaler()),  # Verilerin normalize edilmesi
    ('svm', SVC())  # SVM modeli
])

# Hiperparametre aralıklarını tanımlayın
C_range = np.array([1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02, 1.e+03, 1.e+04, 1.e+05, 1.e+06, 1.e+07, 1.e+08, 1.e+09, 1.e+10])
gamma_range = np.array([1.e-09, 1.e-08, 1.e-07, 1.e-06, 1.e-05, 1.e-04, 1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02, 1.e+03])

param_grid = {
    'svm__C': C_range,
    'svm__gamma': gamma_range
}

# GridSearchCV kullanarak cross validation ile en iyi hiperparametreleri bulma
grid_search = GridSearchCV(pipe, param_grid, cv=10, n_jobs=-1, verbose=2)

# Modeli eğit
grid_search.fit(train_features, train_labels)

# En iyi hiperparametreleri ve sonuçları yazdır
print("En iyi hiperparametreler: ", grid_search.best_params_)
print("En iyi doğruluk skoru: ", grid_search.best_score_)

Fitting 10 folds for each of 169 candidates, totalling 1690 fits
En iyi hiperparametreler:  {'svm__C': 10.0, 'svm__gamma': 0.0001}
En iyi doğruluk skoru:  0.8143570536828963


In [2]:
import numpy as np
import pandas as pd
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, accuracy_score, recall_score, precision_score, f1_score

# Verileri yükleyin
train_features = np.load('train_features_stackedautoencoder.npy')
train_labels = np.load('train_labels_stackedautoencoder.npy')
test_features = np.load('test_features_stackedautoencoder.npy')
test_labels = np.load('test_labels_stackedautoencoder.npy')

# Verileri normalize etmek için StandardScaler kullanın
scaler = StandardScaler()
train_features = scaler.fit_transform(train_features)
test_features = scaler.transform(test_features)

# SVM modelini en iyi hiperparametrelerle oluşturun
svm = SVC(C=10.0, gamma=0.0001)

# Modeli eğit
svm.fit(train_features, train_labels)

# Test seti üzerinde tahminler yapın
test_predictions = svm.predict(test_features)

# Performans metriklerini hesaplayın
accuracy = accuracy_score(test_labels, test_predictions)
recall = recall_score(test_labels, test_predictions)
precision = precision_score(test_labels, test_predictions)
f1 = f1_score(test_labels, test_predictions)

# Sensitivity (Recall) ve Specificity'yi hesaplayın
tn, fp, fn, tp = confusion_matrix(test_labels, test_predictions).ravel()
sensitivity = recall  # Sensitivity, recall ile aynıdır
specificity = tn / (tn + fp)

# Performans metriklerini ekrana yazdır
print(f"Accuracy: {accuracy * 100:.2f}%")
print(f"Recall (Sensitivity): {sensitivity * 100:.2f}%")
print(f"Specificity: {specificity * 100:.2f}%")
print(f"Precision: {precision * 100:.2f}%")
print(f"F1 Score: {f1 * 100:.2f}%")

# Confusion matrix'i hesaplayın ve yazdırın
conf_matrix = confusion_matrix(test_labels, test_predictions)
print("Confusion Matrix:")
print(conf_matrix)

# Tahminleri ve gerçek değerleri bir DataFrame'e ekleyin
df = pd.DataFrame({
    'Gerçek': test_labels,
    'Tahmin': test_predictions
})

# Tahmin ve gerçek değerleri Excel dosyasına kaydedin
df.to_excel('svm_predicts_stackedautoencoder.xlsx', index=False)

# Performans metriklerini ve confusion matrix'i bir dosyaya yazdırmak isterseniz:
with pd.ExcelWriter('svm_model_sonuclari.xlsx') as writer:
    df.to_excel(writer, sheet_name='Tahminler', index=False)
    # Performans metriklerini bir DataFrame'e çevirin
    metrics_df = pd.DataFrame({
        'Metric': ['Accuracy', 'Recall (Sensitivity)', 'Specificity', 'Precision', 'F1 Score'],
        'Value': [accuracy, sensitivity, specificity, precision, f1]
    })
    metrics_df.to_excel(writer, sheet_name='Performans_Metrikleri', index=False)
    
    # Confusion matrix'i de ekleyin
    conf_matrix_df = pd.DataFrame(conf_matrix, index=['Gerçek 0', 'Gerçek 1'], columns=['Tahmin 0', 'Tahmin 1'])
    conf_matrix_df.to_excel(writer, sheet_name='Confusion_Matrix')

print("Tahminler, performans metrikleri ve confusion matrix Excel dosyasına kaydedildi.")

C:\Users\Yasin\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
C:\Users\Yasin\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (


Accuracy: 80.63%
Recall (Sensitivity): 80.72%
Specificity: 80.56%
Precision: 76.14%
F1 Score: 78.36%
Confusion Matrix:
[[174  42]
 [ 32 134]]


C:\Users\Yasin\AppData\Local\Temp/ipykernel_22064/285392917.py:57: UserWarning: Pandas requires version '3.0.5' or newer of 'xlsxwriter' (version '3.0.1' currently installed).
  df.to_excel('svm_predicts_stackedautoencoder.xlsx', index=False)


Tahminler, performans metrikleri ve confusion matrix Excel dosyasına kaydedildi.


C:\Users\Yasin\AppData\Local\Temp/ipykernel_22064/285392917.py:60: UserWarning: Pandas requires version '3.0.5' or newer of 'xlsxwriter' (version '3.0.1' currently installed).
  with pd.ExcelWriter('svm_model_sonuclari.xlsx') as writer:


In [3]:
import pickle

# Modeli kaydet
with open('svm_model_autoencoder.pkl', 'wb') as model_file:
    pickle.dump(svm, model_file)

print("Model başarıyla kaydedildi!")

Model başarıyla kaydedildi!
